In [2]:
from zenpy import Zenpy
import requests
import pandas as pd
import datetime as dt
import json
import time
from urllib.parse import urlencode


In [3]:
# Zenpy accepts an API token
creds = {
    'email' : 'joao.garcia@magazineluiza.com.br',
    'token' : 'fwbsVElXCYh6745XfVg2uu7Nql9M5cufIiRiNhEK',
    'subdomain': 'https://netshoesmarketplace.zendesk.com/'
}

# An OAuth token
'''creds = {
  "subdomain": "https://netshoesmarketplace.zendesk.com/",
  "oauth_token": "fwbsVElXCYh6745XfVg2uu7Nql9M5cufIiRiNhEK"
}'''

# Or a password
'''creds = {
    'email' : 'youremail',
    'password' : 'yourpassword',
    'subdomain': 'yoursubdomain'
}
'''

# Default
zenpy_client = Zenpy(**creds)



In [4]:
zenpy_client.search(type='ticket', status='open')

SSLError: HTTPSConnectionPool(host='https', port=443): Max retries exceeded with url: //netshoesmarketplace.zendesk.com/.zendesk.com/api/v2/search.json?query=type%3Aticket%20status%3Aopen (Caused by SSLError(SSLError(1, '[SSL: SSLV3_ALERT_HANDSHAKE_FAILURE] sslv3 alert handshake failure (_ssl.c:1123)')))

In [ ]:
zenpy_client.search(type='ticket', status_less_than='closed', assignee='fabio.silva@adidas.com', sort_order='desc')

In [ ]:
def export_data(start_date = dt.datetime(2021, 12, 17, tzinfo=dt.timezone.utc)):

  tickets_json = ''
  tickets_df = pd.DataFrame()
  org_df = pd.DataFrame()
  users_df = pd.DataFrame()
  stream_status = False

  while stream_status == False:

    tickets_json = '' #clears the previous api pull 
    result_generator = zenpy_client.tickets.incremental(start_time=start_date, include=['users','organizations','groups'])
    for ticket in result_generator:
      tickets_json += ticket.decode('utf-8', 'ignore') # ticket data comes over as bytes so has to be converted

    df = pd.json_normalize(json.loads(tickets_json)['tickets'])
    tickets_df = pd.concat([tickets_df,df])

    df = pd.json_normalize(json.loads(tickets_json)['organizations'])
    org_df = pd.concat([org_df,df])

    df = pd.json_normalize(json.loads(tickets_json)['users'])
    users_df = pd.concat([users_df,df])

    print(str(len(tickets_df)) +" Tickets Imported, " + str(len(org_df)) + " Orgs Imported, " + str(len(users_df)) + " Users Imported, ")

    start_date = (json.loads(tickets_json)['end_time'])
    stream_status = json.loads(tickets_json)['end_of_stream']
    print("starting {}".format(dt.datetime.fromtimestamp(start_date).strftime("%b-%d-%Y")))
    time.sleep(1)

    if stream_status == True:
      break

  org_df = org_df.drop_duplicates(subset=['id']) #duplicates created from side loading
  users_df = users_df.drop_duplicates(subset=['id'])

  return tickets_df, org_df, users_df

#Run the funciton and export the data as a Pandas DataFrames
tickets_df, org_df, users_df = export_data()

#Merge the data
tickets_users_df = pd.merge(tickets_df, users_df, how='inner', left_on='submitter_id', right_on='id', suffixes=['_tickets','_users'])
tickets_users_orgs_df = pd.merge(tickets_users_df, org_df, how='left', left_on='organization_id_tickets', right_on='id', suffixes=['','_orgs'])
tickets_users_orgs_df.to_csv('tickets_export.csv')
tickets_users_orgs_df.head()